In [2]:
import re
retorno = re.findall("^[0-9]{2}([^@]+)","952345SA#124@APNNE")
retorno

['2345SA#124']

In [1]:
def calculo_daora(x,y):
    if y == 0:
        return 0
    elif y == 1:
        return x
    else : 
        return x +calculo_daora(x,y-1)

valor = calculo_daora(2,500)
valor

1000

In [6]:
import requests
# Request da api na sensedia
url = "https://api.abrasel.com.br/dev.api/v1/estabelecimentos"
client_id = {"client_id": "b78911f9-68ad-4b71-aca8-8dc21237b506"}
params = {
    "cidade":"ARCOS",
    "uf":"MG",
    "associado":"ATIVO",
    "bandeira" :"ticket",
    "pageSize":"50"
}

response = requests.get(url=url, headers=client_id, params=params)

print(response.text)

[{"CNPJ":null,"RAZAO_SOCIAL_RFB":null,"NOME_FANTASIA":"LANCHES DA LUA","CEP":"35588000","ENDERECO":"R DOM MANOEL COELHO NUNES, 23","BAIRRO":"CENTRO","CIDADE":"ARCOS","UF":"MG","TELEFONE":"91337929","TELEFONE_RFB":null,"EMAIL":null,"BANDEIRAS":"TICKET    ","BASE_SIGA":0,"ASSOCIADO":0,"SOU_ABRASEL":0,"LATITUDES":"-19.96953010559082","LONGITUDES":"-44.03247833251953"},{"CNPJ":"18099346000154","RAZAO_SOCIAL_RFB":"BAR E LANCHONETE UNIVERSIDADE CAFE LTDA","NOME_FANTASIA":"BAR E LANCHONETE UNIVERSIDADE CAFE","CEP":"35588000","ENDERECO":"ROD BR 354, 900","BAIRRO":"VILA CALCITA","CIDADE":"ARCOS","UF":"MG","TELEFONE":"99671007","TELEFONE_RFB":"37-99671007","EMAIL":null,"BANDEIRAS":"TICKET    ","BASE_SIGA":0,"ASSOCIADO":0,"SOU_ABRASEL":0,"LATITUDES":"-22.501689910888672","LONGITUDES":"-44.670989990234375"},{"CNPJ":null,"RAZAO_SOCIAL_RFB":null,"NOME_FANTASIA":"SUPERMERCADOS BH","CEP":"35588000","ENDERECO":"AV DOUTOR JOAO VAZ SOBRINHO, 1200","BAIRRO":"VALE DAS MANGUEIRAS","CIDADE":"ARCOS","UF":"MG"